In [1]:
import argparse
import os
import glob
from PIL import Image, ImageDraw
import torch
import cn_clip.clip as clip
import tqdm
import pandas as pd
import matplotlib.pyplot as plt
import time
import os.path
import cv2
import numpy as np
import os
from ultralytics import YOLO
from pathlib import Path
import faiss

WARNING  Python>=3.10 is required, but Python==3.8.20 is currently installed 


In [2]:

plt.rcParams['font.sans-serif'] = ['SimHei']  # 这里设置为支持中文的字体
plt.rcParams['axes.unicode_minus'] = False  # 用于正常显示负号

path = r'E:\bookspines_project\back_py\flaskProject'


# 设置参数
class Args:
    # 图像大小
    input_size = 640
    # yolo最佳模型
    yolo_best_path = os.path.join(path, 'models/best.pt')

    # 搭建库的数据集路径
    database_dir = os.path.join(path, r'database\database_by_yolo')
    # 数据库图片分割后的数据集路径
    database_by_yolo = os.path.join(path, r'segmented_img')
    
    # # 数据库图片分割后数据增强的数据集路径
    # database_dir_aug = r'D:\programme\anaconda3\jupyterfile\guochuang_final\output\database_dir_aug'
    # # 数据库图片分割后的位置路径
    # database_shelf_location = r'D:\programme\anaconda3\jupyterfile\guochuang_final\dataset\location'
    # # 数据库图片分割后的位置信息文件
    # database_shelf_location_csv = r"D:\programme\anaconda3\jupyterfile\guochuang_final\dataset\location\database_shelf_locations2.xlsx"
    # # 数据库图像特征文件路径
    # database_feature_path = r'D:\programme\anaconda3\jupyterfile\guochuang_final\dataset\feature'
    
    # 数据库图像特征文件
    database_feature = os.path.join(path, r'models\corpus_feature_dict.npy')
    # # 数据库索引文件路径
    # database_index_path = r'D:\programme\anaconda3\jupyterfile\guochuang_final\dataset\index'
    # 数据库索引文件
    database_index = os.path.join(path, r'models\faiss_index_ivf.index')

    # 待测图片路径
    test_image_dir = os.path.join(path, r'photos')
    # 待测图片分割完后的路径
    test_image_by_yolo = os.path.join(path, r'segment_by_yolo')
    # # 待测图片的位置信息
    test_shelf_location = os.path.join(path, r'location')
    test_shelf_location_csv = os.path.join(path, r'location\test_shelf_locations.xlsx')
    # 测试图片匹配结果存储路径
    save_dir = os.path.join(path, r'results')

    # 特征提取模型
    model_name = 'clip'  # 可选 resnet50, resnet152, clip

    # 搜寻前k个相似图片
    topk = 2

    # 选择当前模式
    mode = 'extract'  # 'extract' 表示提取特征，'predict' 表示匹配相似图片

    # 匹配算法
    index_type = 'ivf'

    # 边界框信息文件
    bbox_file = os.path.join(path, r'models\bounding_boxes.npy')


args = Args()  # 实例化参数对象

In [3]:


# ## 实例分割：target_extraction_by_yolov8


def target_extraction_by_yolov8(args, datapath, mode=args.mode, batch_size=6):
    if args.mode == 'extract':
        # YOLO模型加载
        model_seg = YOLO(args.yolo_best_path)
        print('YOLO模型加载完毕')

        # 获取所有图像路径
        image_paths = [os.path.join(datapath, img) for img in os.listdir(datapath) if
                       img.endswith(('.jpg', '.png', '.jpeg'))]

        # 将数据分成批次
        total_images = len(image_paths)
        batches = [image_paths[i:i + batch_size] for i in range(0, total_images, batch_size)]

        print(f'总共有 {total_images} 张图片，批次大小为 {batch_size}，将分为 {len(batches)} 个批次进行处理')

        # 用于存储边界框数据的字典
        bbox_data = {}

        for batch_idx, batch in enumerate(batches):
            print(f'正在处理第 {batch_idx + 1} 批次，共 {len(batch)} 张图片')
            print(batch_idx)

            # 批量加载图片进行分割
            results = model_seg(batch)  # 传入当前批次的图片列表
            print('批次实例分割完成')

            # 处理分割结果
            for result in results:
                # 加载原始图像
                image_path = result.path  # Path to the image
                original_image = cv2.imread(image_path)
                image_name = Path(image_path).stem  # 提取文件名，不包括扩展名

                # 获取图像的高度和宽度
                height, width, _ = original_image.shape

                # 检查结果中是否存在掩码
                if result.masks is None:
                    print(f"No masks found for {image_name}. Saving the original image...")
                    continue

                # 迭代每个检测(对象)
                for i, mask in enumerate(result.masks.data):

                    object_name = f"{image_name}_object_{i}"

                    # 获取当前对象的掩码和边界框
                    obj_mask = mask.cpu().numpy()  # Convert mask to NumPy array (H, W)
                    # 将布尔掩码转换为8位无符号整型(OpenCV需要)
                    obj_mask = (obj_mask * 255).astype(np.uint8)

                    # 确保蒙版大小与原始图像大小匹配
                    if obj_mask.shape[0] != height or obj_mask.shape[1] != width:
                        obj_mask = cv2.resize(obj_mask, (width, height), interpolation=cv2.INTER_NEAREST)

                    # 将蒙版直接应用于原始图像以提取对象
                    masked_obj = cv2.bitwise_and(original_image, original_image, mask=obj_mask)

                    # 根据被遮罩对象的边界框裁剪它
                    obj_bbox = result.boxes.xyxy[i].cpu().numpy().astype(
                        int)  # Bounding box in (xmin, ymin, xmax, ymax)
                    xmin, ymin, xmax, ymax = obj_bbox

                    # 存储当前对象的边界框信息，使用 object_name 作为键
                    bbox_data[object_name] = [xmin, ymin, xmax, ymax]

                    cropped_obj = masked_obj[ymin:ymax, xmin:xmax]

                    # 存储分割对象
                    if mode == 'extract':
                        output_path = os.path.join(
                            r'D:\programme\anaconda3\jupyterfile\guochuang_final\output\database_by_yolo_for_boxes',
                            f"{image_name}_object_{i}.jpg")
                    elif mode == 'predict':
                        output_path = os.path.join(args.test_image_by_yolo, f"{image_name}_object_{i}.jpg")

                    cv2.imwrite(output_path, cropped_obj)
                #                 # 存大结果
                result.save(filename=os.path.join(
                    r'D:\programme\anaconda3\jupyterfile\guochuang_final\output\database_by_yolo_BIG2',
                    f"{image_name}_result.jpg"))

            print(f'批次 {batch_idx + 1} 的实例分割结果已存储完毕')
        # 将边界框信息保存到 .npy 文件
        np.save(r'D:\programme\anaconda3\jupyterfile\guochuang_final\dataset\boxes\bounding_boxes.npy', bbox_data)
        print("边界框信息已保存到 bounding_boxes.npy")
    elif args.mode == 'predict':
        # 先用yolo对待测图片进行分割
        model_seg = YOLO(args.yolo_best_path)
        print('模型加载完毕')
        cur_time = time.time()
        results = model_seg(datapath)  # 传入当前批次的图片列表
        # 处理分割结果
        for result in results:
            # 加载原始图像
            image_path = result.path  # Path to the image
            original_image = cv2.imread(image_path)
            image_name = Path(image_path).stem  # 提取文件名，不包括扩展名
            # 获取图像的高度和宽度
            height, width, _ = original_image.shape
            # 检查结果中是否存在掩码
            if result.masks is None:
                print(f"No masks found for {image_name}. Saving the original image...")
                continue

            # 迭代每个检测(对象)
            for i, mask in enumerate(result.masks.data):
                # 获取当前对象的掩码和边界框
                obj_mask = mask.cpu().numpy()  # Convert mask to NumPy array (H, W)
                # 将布尔掩码转换为8位无符号整型(OpenCV需要)
                obj_mask = (obj_mask * 255).astype(np.uint8)
                # 确保蒙版大小与原始图像大小匹配
                if obj_mask.shape[0] != height or obj_mask.shape[1] != width:
                    obj_mask = cv2.resize(obj_mask, (width, height), interpolation=cv2.INTER_NEAREST)
                # 将蒙版直接应用于原始图像以提取对象
                masked_obj = cv2.bitwise_and(original_image, original_image, mask=obj_mask)
                # 根据被遮罩对象的边界框裁剪它
                obj_bbox = result.boxes.xyxy[i].cpu().numpy().astype(int)  # Bounding box in (xmin, ymin, xmax, ymax)
                xmin, ymin, xmax, ymax = obj_bbox
                cropped_obj = masked_obj[ymin:ymax, xmin:xmax]
                # 存储分割对象
                if mode == 'extract':
                    output_path = os.path.join(args.database_by_yolo, f"{image_name}_object_{i}.jpg")
                elif mode == 'predict':
                    output_path = os.path.join(args.test_image_by_yolo, f"{image_name}_object_{i}.jpg")

                cv2.imwrite(output_path, cropped_obj)

            # 存大结果
            result.save(
                filename=os.path.join(r'D:\programme\anaconda3\jupyterfile\guochuang_final\output\test_by_yolo_BIG',
                                      f"{image_name}_result.jpg"))
        print(f'实例分割存储完毕，用时 {time.time() - cur_time:.2f} 秒')
        print('实例分割存储完毕')


def extract_feature_by_CLIP(model, preprocess, file):
    image = preprocess(Image.open(file)).unsqueeze(0).to(device)
    with torch.no_grad():
        vec = model.encode_image(image)
        vec = vec.squeeze(0).cpu().numpy()
    return vec


def load_shelf_locations(file_path):
    df = pd.read_excel(file_path)
    shelf_location = {row['image_path']: row['shelf_location'] for index, row in df.iterrows()}
    return shelf_location


def search_with_faiss(args, model, preprocess, test_images, shelf_location, index_type='ivf'):
    # 加载特征字典以保留位置信息
    # 加载特征数据库
    allVectors = np.load(args.database_feature, allow_pickle=True)
    allVectors = allVectors.item()
    #     print(allVectors.keys())
    # 加载索引
    index_file_name = args.database_index
    index = faiss.read_index(index_file_name)

    # 使用 GPU
    res = faiss.StandardGpuResources()
    index = faiss.index_cpu_to_gpu(res, 0, index)
    #     print(index)
    result = {}
    for img_file in tqdm.tqdm(test_images):
        #         print(img_file)
        if args.model_name == 'clip':
            features = extract_feature_by_CLIP(model, preprocess, img_file)
            features = features / np.linalg.norm(features)  # 归一化

            features = np.array([features])  # 转换为二维数组

            # 使用 FAISS 进行最近邻搜索
            index.nprobe = 1000
            D, I = index.search(features, args.topk)  # 返回 topk 最近邻的索引

            simImages, simScores, simLocations = [], [], []
            for ind in I[0]:  # I[0] 是当前查询的最近邻索引
                # allVectors.keys()是名字
                simImages.append(list(allVectors.keys())[ind])
                simScores.append(D[0][np.where(I[0] == ind)[0][0]])  # 获取对应的距离
                simLocations.append(allVectors[list(allVectors.keys())[ind]]['shelf_location'])
                # print(list(allVectors.keys())[ind], D[0][np.where(I[0] == ind)[0][0]],allVectors[list(allVectors.keys())[ind]]['shelf_location'])
                # print(allVectors)
            # 添加被检测图像的信息，包括位置
            result[img_file] = {
                'query_image': img_file,
                'shelf_location': shelf_location.get(img_file.split('\\')[-1], 'Unknown'),  # 现在从 shelf_location 获取位置信息
                'similar_images': simImages,
                'similar_scores': simScores,
                'similar_locations': simLocations,
            }

    return result


# ### setAxes:绘制相似图片

# In[8]:


def setAxes(ax, image, query=False, **kwargs):
    value = kwargs.get('value', None)
    if query:
        ax.set_xlabel("Query Image\n{0}".format(os.path.basename(image)), fontsize=12)
        ax.xaxis.label.set_color('red')
    else:
        ax.set_xlabel("score={1:1.3f}\n{0}".format(os.path.basename(image), value), fontsize=12)
        ax.xaxis.label.set_color('blue')
    ax.set_xticks([])
    ax.set_yticks([])


def find_shelf_image(result_image_name, shelf_images_dir):
    """
    根据分割结果的文件名，找到对应的书架图片。
    :param result_image_name: 分割结果的文件名
    :param shelf_images_dir: 存放书架图片的目录
    :return: 对应的书架图片路径
    """
    # 去掉 "_object_" 后的前缀，确保精确匹配到该位置
    shelf_image_prefix = result_image_name.split("_object_")[0]

    # 在书架目录中寻找匹配的书架图片
    for img_path in Path(shelf_images_dir).glob(f"{shelf_image_prefix}*.jpg"):
        return str(img_path)  # 返回找到的书架图片路径

    # 如果没有找到，返回 None
    return None


def draw_bounding_box_on_shelf(shelf_image  , bbox):
    """
    在书架图片上绘制书籍的边界框。
    :param shelf_image: PIL Image对象, 书架图片
    :param bbox: 书籍的边界框，格式为 [xmin, ymin, xmax, ymax]
    :return: 绘制了边界框的 PIL Image 对象
    """
    draw = ImageDraw.Draw(shelf_image)
    xmin, ymin, xmax, ymax = bbox
    draw.rectangle([xmin, ymin, xmax, ymax], outline="red", width=45)
    return shelf_image


def load_bounding_boxes(bbox_file):
    """
    加载存储的边界框信息（.npy 文件）
    :param bbox_file: 边界框的 npy 文件路径
    :return: 边界框的字典
    """
    return np.load(bbox_file, allow_pickle=True).item()


def plotSimilarImages(args, image, simImages, simValues, simLocations, numRow=1, numCol=4, query_shelf_location=None):
    """
    绘制查询图像及其相似图像，保持原有功能。
    同时，生成一个新图，展示带有边界框的书架图片，并在下方显示书的位置信息。
    :param args: 参数对象
    :param image: 查询图像路径
    :param simImages: 相似图像的路径列表
    :param simValues: 相似图像的分数列表
    :param simLocations: 相似图像的位置列表
    :param numRow: 行数
    :param numCol: 列数
    :param query_shelf_location: 查询图像的书架位置信息
    """
    # *** 保持原有功能：展示查询图像和相似图像 ***
    fig = plt.figure()
    fig.set_size_inches(18.5, 10.5)
    fig.suptitle(f'use engine model: {args.model_name}', fontsize=35)

    ax = []  # 初始化 ax 列表，用于存储子图

    # 循环展示查询图像及相似图像
    for j in range(0, numCol * numRow):
        if j == 0:
            # 展示查询图像
            img = Image.open(image)
            ax.append(fig.add_subplot(numRow, numCol, 1))
            setAxes(ax[0], image, query=True)

            # 显示被检测图像的位置信息
            if query_shelf_location:
                plt.text(0.5, -0.1, query_shelf_location, ha='center', fontsize=10, transform=ax[0].transAxes)
        else:
            # 展示相似图像
            img = Image.open(simImages[j - 1])
            ax.append(fig.add_subplot(numRow, numCol, j + 1))
            setAxes(ax[j], simImages[j - 1], value=simValues[j - 1])
            plt.imshow(img)

            # 在相似图像下方添加位置信息
            plt.text(0.5, -0.1, simLocations[j - 1], ha='center', fontsize=10, transform=ax[j].transAxes)

        img = img.convert('RGB')
        plt.imshow(img)

        img.close()

    # 获取图片的文件名（不带扩展名）
    image_name = os.path.basename(image).split('.')[0]  # 只保留文件名，不带扩展名

    # 动态创建保存文件夹，根据搜索方法命名文件夹
    search_method_folder = f"{args.save_dir}/{args.index_type}_search_results"
    if not os.path.exists(search_method_folder):
        os.makedirs(search_method_folder)

    # 保存原有展示图像
    save_path = f"{search_method_folder}/_search_top_{args.topk}_{image_name}.png"
    fig.savefig(save_path)
    plt.close(fig)  # 关闭图形以释放内存

    # *** 新增：展示带有边界框的书架图片，添加位置信息 ***
    for j in range(len(simImages)):
        fig2 = plt.figure()
        fig2.set_size_inches(10, 5)
        fig2.suptitle(f'Detected Image and Shelf Image', fontsize=20)

        # 展示相似图片
        ax2_1 = fig2.add_subplot(1, 2, 1)
        img = Image.open(simImages[j])
        ax2_1.set_title("Similar Image")
        ax2_1.set_xticks([])
        ax2_1.set_yticks([])
        plt.imshow(img)

        # 在相似图像下方显示位置信息
        ax2_1.text(0.5, -0.1, simLocations[j], ha='center', fontsize=10, transform=ax2_1.transAxes)

        # 获取书架图片并展示
        result_image_name = Path(simImages[j]).stem  # 根据相似图片获取对应的书架
        shelf_image_path = find_shelf_image(result_image_name, args.database_dir)

        if shelf_image_path:
            print(f"找到的书架图片路径: {shelf_image_path}")  # 调试信息，确认书架图片路径
            shelf_image = Image.open(shelf_image_path).copy()

            bbox_data = load_bounding_boxes(args.bbox_file)
            if result_image_name in bbox_data:
                bbox = bbox_data[result_image_name]
                shelf_image = draw_bounding_box_on_shelf(shelf_image, bbox)
                print(f"应用了边界框: {bbox}")  # 调试信息，确认边界框内容

            # 展示书架图片
            ax2_2 = fig2.add_subplot(1, 2, 2)
            ax2_2.set_title("Shelf Image with Bounding Box")
            ax2_2.set_xticks([])
            ax2_2.set_yticks([])
            plt.imshow(shelf_image)

            # 在书架图片下方显示位置信息
            ax2_2.text(0.5, -0.1, simLocations[j], ha='center', fontsize=10, transform=ax2_2.transAxes)
        else:
            print("未找到书架图片。")  # 调试信息，确认找不到书架图片时的情况

        # 保存包含带有边界框书架图片的图像
        marked_shelf_folder = os.path.join(args.save_dir, "marked_shelves")
        if not os.path.exists(marked_shelf_folder):
            os.makedirs(marked_shelf_folder)

        save_marked_path = os.path.join(marked_shelf_folder, f"{result_image_name}_shelf_combined.png")
        fig2.savefig(save_marked_path)
        print(f"已保存图像: {save_marked_path}")
        plt.close(fig2)  # 关闭图形以释放内存


def getSimilarMatrix(vectors_dict):
    v = np.array([value['features'] for value in vectors_dict.values()])  # 确保提取特征
    numerator = np.matmul(v, v.T)  # 分子
    denominator = np.matmul(np.linalg.norm(v, axis=1, keepdims=True), np.linalg.norm(v, axis=1, keepdims=True).T)  # 分母
    sim = numerator / denominator
    keys = list(vectors_dict.keys())
    # print(sim)

    return sim, keys


def baoli(args):
    allVectors = np.load(args.database_feature, allow_pickle=True)
    allVectors = allVectors.item()

    for img_file in tqdm.tqdm(test_images):
        if args.model_name == 'clip':
            features = extract_feature_by_CLIP(model, preprocess, img_file)
            # 添加位置信息，默认为"Unknown"
            shelf_loc = shelf_location.get(img_file, 'Unknown') if shelf_location else 'Unknown'
            allVectors[img_file] = {
                'features': features,
                'shelf_location': shelf_loc
            }

    sim, keys = getSimilarMatrix(allVectors)
    result = {}
    # print('hhh')
    for image_file in tqdm.tqdm(test_images):
        # print(image_file)
        # print(f'sorting most similar images as {image_file}')

        index = keys.index(image_file)
        sim_vec = sim[index]
        # 获取前 topk + 1 个相似图片（包括自身）
        indexs = np.argsort(sim_vec)[::-1]  # 先按相似度排序，从大到小
        simImages, simScores, simLocations = [], [], []
        for ind in indexs:
            if keys[ind] != image_file:  # 过滤掉被检测的图片
                simImages.append(keys[ind])
                simScores.append(sim_vec[ind])
                simLocations.append(allVectors[keys[ind]]['shelf_location'])  # 获取位置信息
            if len(simImages) >= args.topk:  # 确保只获取 topk 个相似图像
                break

        result[image_file] = (simImages, simScores, simLocations)  # 将位置信息也存入结果
    return result

In [4]:

from cn_clip.clip import load_from_name, available_models

# device = 'cuda' if torch.cuda.is_available() else 'cpu'
# model, preprocess = clip.load('ViT-L/14@336px', device=device)  # 加载 CLIP 模型
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = load_from_name("ViT-H-14", device=device, download_root='./')

args.mode = 'predict'
print(args.mode)

Loading vision model config from F:\miniconda\envs\flaskProject\lib\site-packages\cn_clip\clip\model_configs\ViT-H-14.json
Loading text model config from F:\miniconda\envs\flaskProject\lib\site-packages\cn_clip\clip\model_configs\RoBERTa-wwm-ext-large-chinese.json
Model info {'embed_dim': 1024, 'image_resolution': 224, 'vision_layers': 32, 'vision_width': 1280, 'vision_head_width': 80, 'vision_patch_size': 14, 'vocab_size': 21128, 'text_attention_probs_dropout_prob': 0.1, 'text_hidden_act': 'gelu', 'text_hidden_dropout_prob': 0.1, 'text_hidden_size': 1024, 'text_initializer_range': 0.02, 'text_intermediate_size': 4096, 'text_max_position_embeddings': 512, 'text_num_attention_heads': 16, 'text_num_hidden_layers': 24, 'text_type_vocab_size': 2}
predict


In [7]:

args.mode = 'predict'
# 第二阶段：图像检索
if args.mode == 'predict':
    start_time_extract = time.time()  # 记录开始时间

    target_extraction_by_yolov8(args,
                                os.path.join(args.test_image_dir, 'IMG_20241013_213453_edit_521718209048478.jpg'),
                                'predict')
    print(f'用 clip 模型来提取特征')

    end_time_extract = time.time()  # 记录结束时间
    print(f"Time taken to extract features: {end_time_extract - start_time_extract:.2f} seconds")

# ## 生成位置信息


模型加载完毕

image 1/1 E:\bookspines_project\back_py\flaskProject\photos\IMG_20241013_213453_edit_521718209048478.jpg: 352x640 35 bookspines, 97.6ms
Speed: 3.0ms preprocess, 97.6ms inference, 5.0ms postprocess per image at shape (1, 3, 352, 640)
实例分割存储完毕，用时 2.23 秒
实例分割存储完毕
用 clip 模型来提取特征
Time taken to extract features: 2.39 seconds


In [8]:

args.mode = 'predict'
if args.mode == 'predict':
    start_time_match = time.time()  # 记录开始时间

    print(f'use preyrained model {args.model_name} to search {args.topk} similar images from corpus')

    # 加载测试图片
    test_images = glob.glob(os.path.join(args.test_image_by_yolo, '*.jpg'))

    # 在主函数中加载位置信息
    shelf_location = {}
    if args.test_shelf_location:  # 检查是否提供了路径
        if os.path.exists(args.test_shelf_location_csv):
            shelf_location = load_shelf_locations(args.test_shelf_location_csv)
        else:
            print("Warning: Text shelf location file not found.")
    else:
        print("Warning: No text shelf location file provided.")

    args.index_type = 'ivf'
    # 使用索引进行搜索，传递 shelf_location 参数
    if args.index_type == 'ivf':
        result = search_with_faiss(args, model, preprocess, test_images, shelf_location, index_type='ivf')  # 或 'flat'

        #         # 生成测试图片的位置信息CSV
        #         generate_test_location_csv(args, result)

        for img_file in test_images:
            query_result = result.get(img_file, None)
            if query_result:
                plotSimilarImages(args, query_result['query_image'], query_result['similar_images'],
                                  query_result['similar_scores'], query_result['similar_locations'],
                                  numRow=1, numCol=args.topk + 1, query_shelf_location=query_result['shelf_location'])
    elif args.index_type == 'flat':
        result = baoli(args)
        for image_file in test_images:
            # 在调用 plotSimilarImages 的地方
            plotSimilarImages(args, image_file, result[image_file][0], result[image_file][1], result[image_file][2],
                              numRow=1, numCol=args.topk + 1, query_shelf_location=shelf_location.get(image_file, None))

    end_time_match = time.time()  # 记录结束时间
    print(f"Time taken to match images: {end_time_match - start_time_match:.2f} seconds")


use preyrained model clip to search 2 similar images from corpus


100%|██████████| 52/52 [00:07<00:00,  6.63it/s]


找到的书架图片路径: E:\bookspines_project\back_py\flaskProject\database\database_by_yolo\IMG_20241013_172727_edit_509459514711800_scaled_small_object_0.jpg
应用了边界框: [2564, 212, 2821, 2556]
已保存图像: E:\bookspines_project\back_py\flaskProject\results\marked_shelves\IMG_20241013_172727_edit_509459514711800_scaled_small_object_6_shelf_combined.png
找到的书架图片路径: E:\bookspines_project\back_py\flaskProject\database\database_by_yolo\IMG_20241013_172727_edit_509459514711800_enhanced_1_object_0.jpg
应用了边界框: [2972, 247, 3275, 2954]
已保存图像: E:\bookspines_project\back_py\flaskProject\results\marked_shelves\IMG_20241013_172727_edit_509459514711800_object_2_shelf_combined.png
找到的书架图片路径: E:\bookspines_project\back_py\flaskProject\database\database_by_yolo\IMG_20241013_172727_edit_509459514711800_enhanced_1_object_0.jpg
应用了边界框: [444, 505, 671, 2989]
已保存图像: E:\bookspines_project\back_py\flaskProject\results\marked_shelves\IMG_20241013_172727_edit_509459514711800_object_7_shelf_combined.png
找到的书架图片路径: E:\bookspines_proje

In [23]:
# # 从npy文件中加载数据
# data = np.load('models/corpus_feature_dict.npy', allow_pickle=True)
# 
# # 定义要替换的旧路径和新路径
# old_path_segment = 'D:\\programme\\anaconda3\\jupyterfile\\guochuang_final\\output\\database_by_yolo2\\'
# new_path_segment = 'E:\\bookspines_project\\back_py\\flaskProject\\database\\database_by_yolo\\'
# # 创建一个新的字典来存储修改后的数据
# updated_data = {}
# if isinstance(data, np.ndarray) and data.ndim == 0:
#     # 如果是零维数组，提取内容
#     data = data.item()  # 获取字典
# # 确保数据是字典
# if isinstance(data, dict):
#     print(11)
#     updated_data = {}
#     for key,value in data.items():
#         # 修改键
#         new_key = key.replace(old_path_segment, new_path_segment)
#         # 将新键和旧值添加到新字典中
#         updated_data[new_key] = value
#     np.save('models/corpus_feature_dict.npy', updated_data)

46624


In [2]:
import numpy as np
data = np.load('models/corpus_feature_dict.npy', allow_pickle=True)
data = data.item()
data['E:\\bookspines_project\\back_py\\flaskProject\\database\\database_by_yolo\\IMG_20241013_172727_edit_509459514711800_scaled_small_object_6.jpg']


{'features': array([ 0.01315 ,  0.002583, -0.02711 , ..., -0.000293, -0.01721 ,
         0.04813 ], dtype=float16),
 'shelf_location': '五楼南区三十五排二架二行'}

In [3]:
import os

file_path = 'E:\\bookspines_project\\back_py\\flaskProject\\database\\database_by_yolo\\IMG_20241013_172727_edit_509459514711800_scaled_small_object_6.jpg'
if not os.path.exists(file_path):
    print("文件不存在:", file_path)
else:
    print("文件存在:", file_path)

文件不存在: E:\bookspines_project\back_py\flaskProject\database\database_by_yolo\IMG_20241013_172727_edit_509459514711800_scaled_small_object_6.jpg
